# 01. file read, preprocessing

In [27]:
# 01. file read, preprocessingimport re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [28]:
# 엑셀 파일 읽어오기
import pandas as pd
import numpy as np
df = pd.read_excel('crawling_data_all.xlsx')
df

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Document Type,Publication Stage,Open Access,Source,EID,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,"Jouin C., Mlocek Z.",57219668257;57219668269;,Evolutions of nature based solutions for storm...,2020,Techniques - Sciences - Methodes,115,3,NaN,61.0,70.0,...,Article,Final,NaN,Scopus,2-s2.0-85094632803,NaN,NaN,NaN,NaN,NaN
1,Buzási A.,56692966000;,Climate vulnerability and adaptation challenge...,2021,Climate,9,2,25,1.0,17.0,...,Article,Final,NaN,Scopus,2-s2.0-85100598191,NaN,NaN,NaN,NaN,NaN
2,"Yu G.-R., Yang M., Chen Z., Zhang L.-M.",55807480400;55703268000;57001631400;55077923500;,Technical approach and strategic plan for larg...,2021,Chinese Journal of Applied Ecology,32,4,NaN,1141.0,1153.0,...,Article,Final,NaN,Scopus,2-s2.0-85104538541,NaN,NaN,NaN,NaN,NaN
3,"Thorn J.P.R., Biancardi Aleu R., Wijesinghe A....",56613034600;57260808600;57226747891;5726048210...,Mainstreaming nature-based solutions for clima...,2021,Landscape and Urban Planning,216,NaN,104235,NaN,NaN,...,Article,Final,NaN,Scopus,2-s2.0-85115004828,NaN,NaN,NaN,NaN,NaN
4,"Schuerch M., Spencer T., Temmerman S., Kirwan ...",54783506900;55156903100;6602902860;10044268300...,Future response of global coastal wetlands to ...,2018,Nature,561,7722,NaN,231.0,234.0,...,Article,Final,NaN,Scopus,2-s2.0-85053248555,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,Antoušková M.,42161107100;,Economic value of recreation - Determinants in...,2012,Agris On-line Papers in Economics and Informatics,4,SUPPL.SPL. 4,NaN,3.0,9.0,...,Article,Final,NaN,Scopus,2-s2.0-84874044123,NaN,NaN,NaN,NaN,NaN
1052,Nicol R.,55173678200;,Entering the Fray: The role of outdoor educati...,2014,Educational Philosophy and Theory,46,5,NaN,449.0,461.0,...,Article,Final,NaN,Scopus,2-s2.0-84899996845,NaN,NaN,NaN,NaN,NaN
1053,"Boller F., Hunziker M., Conedera M., Elsasser ...",36647235100;6701797579;6603211787;7005909792;8...,Fascinating remoteness: The dilemma of hiking ...,2010,Mountain Research and Development,30,4,NaN,320.0,331.0,...,Article,Final,NaN,Scopus,2-s2.0-78649521966,NaN,NaN,NaN,NaN,NaN
1054,"Vedel S.E., Thorsen B.J., Jacobsen J.B.",15057304900;6701749452;35773326500;,"First-movers, non-movers, and social gains fro...",2009,Ecological Economics,68,2021-08-09 00:00:00,NaN,2363.0,2371.0,...,Article,Final,NaN,Scopus,2-s2.0-65649085241,NaN,NaN,NaN,NaN,NaN


In [29]:
# 초록이 없는 것 조회
df[df['Abstract'].isnull()]

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Document Type,Publication Stage,Open Access,Source,EID,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28


In [30]:
# 요약문의 형식을 살피기
df.Abstract[11]

'Inadequate sanitation infrastructure is a global problem that is particularly impacting rural areas. And decentralized wastewater management system is considered as the feasible solution for rural sewage treatment (RuST). However, determining the cost-effective (CE) pattern for decentralized RuST is methodologically challenging because of scarce decision-support tools. In this research, a RuST optimization model (RuST-OM) was developed to gain an insight into the CE pattern of RuST based on the greedy algorithm. This model involves tradeoffs in the economy-of-scale and technology of wastewater treatment system versus the cost and energy consumption of the sewage collection system. The investment associated with the CE pattern for RuST is closely linked to the environmental demand, RuST coverage, topographic complexity, and degree of household dispersion. The cost of the CE pattern falls between the onsite-B and community-based pattern, and this range represents the optimized interval 

In [31]:
# 요약문에서 실제로 내용이 있는 부분('@' 앞)만을 가져와, 새 컬럼 Abs_split에 저장
abs_split = df['Abstract'].str.split('©')
df['Abs_split'] = abs_split.str.get(0)
df.Abs_split[11]

'Inadequate sanitation infrastructure is a global problem that is particularly impacting rural areas. And decentralized wastewater management system is considered as the feasible solution for rural sewage treatment (RuST). However, determining the cost-effective (CE) pattern for decentralized RuST is methodologically challenging because of scarce decision-support tools. In this research, a RuST optimization model (RuST-OM) was developed to gain an insight into the CE pattern of RuST based on the greedy algorithm. This model involves tradeoffs in the economy-of-scale and technology of wastewater treatment system versus the cost and energy consumption of the sewage collection system. The investment associated with the CE pattern for RuST is closely linked to the environmental demand, RuST coverage, topographic complexity, and degree of household dispersion. The cost of the CE pattern falls between the onsite-B and community-based pattern, and this range represents the optimized interval 

In [32]:
#초록과 저자 키워드 있는 것  -> 새로운 컬럼 abs+key에 합치기 
df["abs+key"]= df["Abs_split"] +" "+df["Index Keywords"]
    
##한슬언니한테 물어보기 

df["abs+key"] = df["abs+key"].fillna(df["Abs_split"])
df["abs+key"]

0       Stormwater management is one of the greatest c...
1       Wine producers face several challenges regardi...
2       Terrestrial ecosystems are the core components...
3       (Despite a growing recognition of the importan...
4       The response of coastal wetlands to sea-level ...
                              ...                        
1051    The presented article deals with an indirect e...
1052    This article draws on different bodies of know...
1053    Remote areas devoid of roads and tourist trans...
1054    The importance of ecosystem services for manki...
1055    Soil moisture content (0–90 cm depth) and nitr...
Name: abs+key, Length: 1056, dtype: object

In [33]:
#df_final 저장하기 'Unnamed:24', 'Unnamed:25','Unnamed:26','Unnamed:27','Unnamed:28'
# 필요없는 column drop
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_final= df.drop(['Volume',	'Issue',	'Art. No.',	'Page start',	'Page end',	'Page count','Cited by','DOI','Link','Author(s) ID','Abstract', 'Author Keywords', 'Index Keywords', 'Abs_split','Document Type','Open Access','EID',],axis=1)

In [34]:
df_final

,Authors,Title,Year,Source title,Affiliations,Authors with affiliations,Publication Stage,Source,abs+key
0,"Jouin C., Mlocek Z.",Evolutions of nature based solutions for storm...,2020,Techniques - Sciences - Methodes,"Atelier Ld, Paris, France","Jouin, C., Atelier Ld, Paris, France; Mlocek, ...",Final,Scopus,Stormwater management is one of the greatest c...
1,Buzási A.,Climate vulnerability and adaptation challenge...,2021,Climate,"Department of Environmental Economics, Budapes...","Buzási, A., Department of Environmental Econom...",Final,Scopus,Wine producers face several challenges regardi...
2,"Yu G.-R., Yang M., Chen Z., Zhang L.-M.",Technical approach and strategic plan for larg...,2021,Chinese Journal of Applied Ecology,Synthesis Research Center of Chinese Ecosystem...,"Yu, G.-R., Synthesis Research Center of Chines...",Final,Scopus,Terrestrial ecosystems are the core components...
3,"Thorn J.P.R., Biancardi Aleu R., Wijesinghe A....",Mainstreaming nature-based solutions for clima...,2021,Landscape and Urban Planning,"African Climate and Development Initiative, Un...","Thorn, J.P.R., African Climate and Development...",Final,Scopus,(Despite a growing recognition of the importan...
4,"Schuerch M., Spencer T., Temmerman S., Kirwan ...",Future response of global coastal wetlands to ...,2018,Nature,"Lincoln Centre for Water and Planetary Health,...","Schuerch, M., Lincoln Centre for Water and Pla...",Final,Scopus,The response of coastal wetlands to sea-level ...
...,...,...,...,...,...,...,...,...,...
1051,Antoušková M.,Economic value of recreation - Determinants in...,2012,Agris On-line Papers in Economics and Informatics,"Faculty of Economics and Management, Czech Uni...","Antoušková, M., Faculty of Economics and Manag...",Final,Scopus,The presented article deals with an indirect e...
1052,Nicol R.,Entering the Fray: The role of outdoor educati...,2014,Educational Philosophy and Theory,"School of Education, University of Edinburgh, ...","Nicol, R., School of Education, University of ...",Final,Scopus,This article draws on different bodies of know...
1053,"Boller F., Hunziker M., Conedera M., Elsasser ...",Fascinating remoteness: The dilemma of hiking ...,2010,Mountain Research and Development,"Department of Geography, University of Zurich,...","Boller, F., Department of Geography, Universit...",Final,Scopus,Remote areas devoid of roads and tourist trans...
1054,"Vedel S.E., Thorsen B.J., Jacobsen J.B.","First-movers, non-movers, and social gains fro...",2009,Ecological Economics,"Forest and Landscape, University of Copenhagen...","Vedel, S.E., Forest and Landscape, University ...",Final,Scopus,The importance of ecosystem services for manki...


# 02. 영문 형태소 분석 using nltk

In [35]:
documents = list(df_final['abs+key'])
print(len(documents))
documents[0]

1056


"Stormwater management is one of the greatest challenges in the modern urban environment. Doing it well – in a way which respects the natural environment and responds to its constraints – allows to address a multitude of other issues: social (creating amenity space, rebuilding connections between people and nature and improving the living environment), economical (increasing the land value, reducing development and maintenance costs), environmental (ecological corridors, biodiversity, mitigating heath island effects)… Urban designers have got an array of tools to use in order to integrate natural water management into city projects. The aim of this article is, using a number of case study projects in Villeneuve-la-Garenne, to present some of the most commonly used, efficient and simple to implement solutions which can be used to integrate NBS (Nature Based Solutions) into the stormwater management of a city at an individual project scale. The case studies will also provide a starting p

## Preprocessing

In [36]:
#preprocessing 
import re 
import nltk 
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import preprocess_string

nltk.download('stopwords')
stop_words = stopwords.words('english')

def clean_text(d):
    pattern = r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', d)
    return d

def clean_stopword(d):
    return ' '.join([w.lower() for w in d.split() if w not in stop_words and len(w) > 3])

def preprocessing(d):
    return preprocess_string(d)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ra4\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
df_final['abs+key']=df['abs+key'].apply(clean_text)
df_final['abs+key']

0       Stormwater management is one of the greatest c...
1       Wine producers face several challenges regardi...
2       Terrestrial ecosystems are the core components...
3       (Despite a growing recognition of the importan...
4       The response of coastal wetlands to sea-level ...
                              ...                        
1051    The presented article deals with an indirect e...
1052    This article draws on different bodies of know...
1053    Remote areas devoid of roads and tourist trans...
1054    The importance of ecosystem services for manki...
1055    Soil moisture content (0–90 cm depth) and nitr...
Name: abs+key, Length: 1056, dtype: object

In [38]:
df_final['abs+key']=df['abs+key'].apply(clean_stopword)
df_final['abs+key']

0       stormwater management greatest challenges mode...
1       wine producers face several challenges regardi...
2       terrestrial ecosystems core components biosphe...
3       (despite growing recognition importance design...
4       response coastal wetlands sea-level rise twent...
                              ...                        
1051    presented article deals indirect economic valu...
1052    this article draws different bodies knowledge ...
1053    remote areas devoid roads tourist transport in...
1054    importance ecosystem services mankind stresses...
1055    soil moisture content (0–90 depth) nitrate-nit...
Name: abs+key, Length: 1056, dtype: object

## Tokenization

In [39]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [40]:
#word_tokenize-> '기준으로 다 나눠버림 
#WordPunctTokenizer -> 구두점을 별도로 분류하는 특징 
#text_to_word_sequence() => 모든 알파벳을 소문자로 바꾸면서 마침표, 컴마, 느낌표 제거 아포스트로피  보존 


# initializing list
documents
  
# printing original list
print("The original list : " + str(documents[0]))
  
# using list comprehension + split()
# Tokenizing strings in list of strings
tokenized_abs = [text_to_word_sequence(sub) for sub in documents]
  
# print result
print("The list after split of strings is : " + str(tokenized_abs[0]))

##df_final에 tokenized된 abstract + key list 형태로 달기 
df_final['tokenized_abs'] = np.array(tokenized_abs)




print(df_final['abs+key'])
print(df_final['tokenized_abs'])

The original list : Stormwater management is one of the greatest challenges in the modern urban environment. Doing it well – in a way which respects the natural environment and responds to its constraints – allows to address a multitude of other issues: social (creating amenity space, rebuilding connections between people and nature and improving the living environment), economical (increasing the land value, reducing development and maintenance costs), environmental (ecological corridors, biodiversity, mitigating heath island effects)… Urban designers have got an array of tools to use in order to integrate natural water management into city projects. The aim of this article is, using a number of case study projects in Villeneuve-la-Garenne, to present some of the most commonly used, efficient and simple to implement solutions which can be used to integrate NBS (Nature Based Solutions) into the stormwater management of a city at an individual project scale. The case studies will also p

C:\Users\ra4\AppData\Local\Temp\ipykernel_1700\161732609.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df_final['tokenized_abs'] = np.array(tokenized_abs)


# Gensim을 이용한 Topic Modeling

In [41]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(df_final['tokenized_abs'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[df_final['tokenized_abs']], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[df_final['tokenized_abs'][0]]])
print("tokenized_abs개수",len(df_final['tokenized_abs'][0]))
print("trigam+mod 개수",len(trigram_mod[bigram_mod[df_final['tokenized_abs'][0]]]))

['stormwater', 'management', 'is', 'one', 'of', 'the', 'greatest', 'challenges', 'in', 'the', 'modern', 'urban', 'environment', 'doing', 'it', 'well', '–', 'in', 'a', 'way', 'which', 'respects', 'the', 'natural', 'environment', 'and', 'responds', 'to', 'its', 'constraints', '–', 'allows', 'to', 'address', 'a', 'multitude', 'of', 'other', 'issues', 'social', 'creating', 'amenity', 'space', 'rebuilding', 'connections', 'between', 'people', 'and', 'nature', 'and', 'improving', 'the', 'living', 'environment', 'economical', 'increasing', 'the', 'land', 'value', 'reducing', 'development', 'and', 'maintenance', 'costs', 'environmental', 'ecological', 'corridors', 'biodiversity', 'mitigating', 'heath', 'island', 'effects', '…', 'urban', 'designers', 'have', 'got', 'an', 'array', 'of', 'tools', 'to', 'use', 'in', 'order', 'to', 'integrate', 'natural', 'water', 'management', 'into', 'city', 'projects', 'the', 'aim', 'of', 'this', 'article', 'is', 'using', 'a', 'number', 'of', 'case_study', 'proj

# 9. Creating Bigram and Trigram Models

In [42]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# 10. Remove Stopwords, Make Bigrams and Lemmatize

In [43]:


# Remove Stop Words
data_words_nostops = remove_stopwords(df_final['tokenized_abs'])

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


print(data_lemmatized[:1])

[['stormwater', 'management', 'great', 'challenge', 'modern', 'urban', 'environment', 'well', 'way', 'respect', 'natural', 'environment', 'respond', 'constraint', 'allow', 'address', 'multitude', 'issue', 'social', 'create', 'amenity', 'space', 'rebuilding', 'connection', 'people', 'nature', 'improve', 'live', 'environment', 'economical', 'increase', 'land', 'value', 'reduce', 'development', 'maintenance', 'cost', 'environmental', 'ecological', 'corridor', 'biodiversity', 'mitigate', 'heath', 'island', 'effect', 'urban', 'designer', 'get', 'array', 'tool', 'use', 'order', 'integrate', 'natural', 'water', 'management', 'city', 'project', 'aim', 'article', 'use', 'number', 'case', 'study', 'project', 'present', 'commonly_use', 'efficient', 'simple', 'implement', 'solution', 'use', 'integrate', 'nature', 'base', 'solution', 'stormwater', 'management', 'city', 'individual', 'project', 'scale', 'case_studie', 'also', 'provide', 'start', 'point', 'discussion', 'typical', 'obstacle', 'face', 

# 11. Create the Dictionary and Corpus needed for Topic Modeling

In [44]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 3), (27, 3), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 3), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 3), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 3), (61, 7), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 2), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 4), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 2), (103, 2), (104, 1), (105, 1), (106, 4), (107, 1), (108, 2), (109, 1), (110, 1)

In [45]:
id2word[0]

'address'

In [46]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('address', 1),
  ('aim', 1),
  ('allow', 1),
  ('also', 1),
  ('amenity', 1),
  ('analysis', 1),
  ('apply', 1),
  ('array', 1),
  ('article', 1),
  ('assessment', 1),
  ('back', 1),
  ('base', 1),
  ('become', 1),
  ('biodiversity', 1),
  ('case', 1),
  ('case_studie', 1),
  ('challenge', 1),
  ('city', 2),
  ('commonly_use', 1),
  ('connection', 1),
  ('constraint', 1),
  ('corridor', 1),
  ('cost', 2),
  ('cost_benefit', 1),
  ('create', 1),
  ('deal', 1),
  ('design', 3),
  ('designer', 3),
  ('development', 1),
  ('discussion', 1),
  ('ecological', 1),
  ('economical', 1),
  ('effect', 1),
  ('efficiency', 1),
  ('efficient', 1),
  ('encourage', 1),
  ('endorsed', 1),
  ('environment', 3),
  ('environmental', 1),
  ('essential', 1),
  ('exist', 1),
  ('face', 1),
  ('feedback', 1),
  ('get', 1),
  ('great', 1),
  ('heath', 1),
  ('however', 1),
  ('implement', 2),
  ('improve', 1),
  ('increase', 1),
  ('individual', 1),
  ('infiltrate', 1),
  ('integrate', 3),
  ('island', 1),

# 12. Building the Topic Model

In [51]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# 13. View the topics in LDA model

In [52]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

"""How to interpret this?

Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic."""

[(0,
  '0.027*"plant" + 0.026*"concentration" + 0.020*"pollutant" + 0.020*"removal" '
  '+ 0.019*"pollution" + 0.017*"nitrogen" + 0.013*"treatment" + 0.013*"pm" + '
  '0.013*"roof" + 0.011*"nutrient"'),
 (1,
  '0.043*"climate" + 0.041*"change" + 0.022*"risk" + 0.022*"flood" + '
  '0.015*"increase" + 0.013*"base" + 0.013*"management" + 0.013*"adaptation" + '
  '0.012*"land" + 0.011*"impact"'),
 (2,
  '0.039*"coastal" + 0.026*"sediment" + 0.019*"wave" + 0.019*"wetland" + '
  '0.013*"erosion" + 0.012*"restoration" + 0.010*"fish" + 0.010*"protection" + '
  '0.009*"dam" + 0.008*"dune"'),
 (3,
  '0.094*"water" + 0.022*"system" + 0.014*"quality" + 0.013*"runoff" + '
  '0.013*"flow" + 0.012*"river" + 0.012*"use" + 0.012*"management" + '
  '0.010*"stormwater" + 0.010*"wetland"'),
 (4,
  '0.026*"urban" + 0.019*"ecosystem" + 0.018*"nature" + 0.018*"base" + '
  '0.015*"solution" + 0.014*"service" + 0.012*"environmental" + '
  '0.012*"approach" + 0.012*"city" + 0.010*"planning"'),
 (5,
  '0.037*"so

'How to interpret this?\n\nTopic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.\n\nIt means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.\n\nThe weights reflect how important a keyword is to that topic.'

# 14. Compute Model Perplexity and Coherence Score

In [53]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.525514417544038

Coherence Score:  0.4234663622429925


# 15. Visualize the topics-keywords

In [54]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\ra4\anaconda3\envs\dilab\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.218911  0.056333       1        1  38.474518
1     -0.184452 -0.008755       2        1  16.009651
5     -0.078129 -0.023196       3        1  11.922205
3     -0.071757 -0.141089       4        1  11.564515
8     -0.066375 -0.029950       5        1   8.355783
0      0.130230 -0.176475       6        1   5.393401
2      0.223569 -0.009830       7        1   4.458329
6      0.054555  0.308663       8        1   3.578269
7      0.211269  0.024299       9        1   0.243329, topic_info=          Term         Freq        Total Category  logprob  loglift
115      urban  2397.000000  2397.000000  Default  30.0000  30.0000
118      water  1789.000000  1789.000000  Default  29.0000  29.0000
17        city   920.000000   920.000000  Default  28.0000  28.0000
134    climate  1276.000000  1276.000000  Default  27.0000  27.0000
132     change  1293.000000  1293.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
5402      weed     2.083633     8.053562   Topic9  -5.2613   4.6665
464    erosion    21.208476   130.868353   Topic9  -2.9410   4.1987
5768       row     1.014728     5.227453   Topic9  -5.9808   4.3792
1416  rainfall     2.414638   120.466594   Topic9  -5.1139   2.1087
432   sediment     2.025742   194.484565   Topic9  -5.2895   1.4541

[544 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
122       1  0.058870  adaptation
122       2  0.939119  adaptation
0         1  0.970302     address
0         2  0.025094     address
1337      6  0.974790  adsorption
...     ...       ...         ...
523       3  0.572366        year
523       4  0.253342        year
523       7  0.084447        year
1224      3  0.895062       yield
1224      7  0.095219       yield

[916 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 6, 4, 9, 1, 3, 7, 8])